In [11]:
import logging
import os.path
import argparse
from typing import Any, Union

# External modules
import xarray as xr
import numpy as np
from tqdm import tqdm

# Internal modules

def get_regridder(
        grid_in: xr.Dataset, out_res: float, method='bilinear'
) -> Any:
    import xesmf as xe
    grid_out = xr.Dataset(
        {
            'lat': (['lat'], np.arange(-90+out_res/2, 90, out_res)),
            'lon': (['lon'], np.arange(0, 360, out_res)),
        }
    )
    regridder = xe.Regridder(grid_in, grid_out, method=method,
                             periodic=True, reuse_weights=False)
    return regridder


def _regrid_loop(
        regridder: Any,
        ds: xr.Dataset,
        chunk_size: Union[int, None] = None
) -> xr.Dataset:
    if chunk_size is None:
        chunk_size = len(ds['time'])
    ds_regridded = []
    regrid_pbar = tqdm(range(0, len(ds['time']), chunk_size))
    for chunk in regrid_pbar:
        tmp_regridded = regridder(
            ds.isel(time=slice(chunk, chunk+chunk_size))
        )
        ds_regridded.append(tmp_regridded)
    ds_regridded = xr.concat(ds_regridded, dim='time')
    for var in ds_regridded:
        ds_regridded[var].attrs = ds[var].attrs
    ds_regridded.attrs = ds.attrs
    return ds_regridded


def regrid_ds(
        ds_ens: Union[xr.Dataset, None],
        chunk_size: Union[int, None] = None
) -> Union[xr.Dataset, None]:
    if ds_ens is not None:
        ds_ens = ds_ens.rename({'latitude': 'lat', 'longitude': 'lon'})
        grid_in = ds_ens[['lat', 'lon']]
        regridder = get_regridder(grid_in, out_res=5.625)
        ds_regridded = _regrid_loop(regridder, ds_ens, chunk_size)
    else:
        ds_regridded = None
    return ds_regridded

In [12]:
import xarray as xr
import distributed


In [13]:
cluster = distributed.LocalCluster(n_workers=4, threads_per_worker=1, local_directory='/tmp')
client = distributed.Client(cluster)
client

c:\Users\haris\miniconda3\envs\ET\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 57290 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:57290/status,
Dashboard: http://127.0.0.1:57290/status,Workers: 4
Total threads: 4,Total memory: 15.34 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:57291,Workers: 4
Dashboard: http://127.0.0.1:57290/status,Total threads: 4
Started: Just now,Total memory: 15.34 GiB
Comm: tcp://127.0.0.1:57315,Total threads: 1
Dashboard: http://127.0.0.1:57318/status,Memory: 3.84 GiB
Nanny: tcp://127.0.0.1:57294,


# Load IFS data

In [14]:
ds_ifs_pl = xr.open_dataset(r'D:\Projects\NIOT\v1\ensemble_transformer\scripts\data\delhi\pl.nc',chunks={'time': 2})
ds_ifs_sfc = xr.open_dataset(r'D:\Projects\NIOT\v1\ensemble_transformer\scripts\data\delhi\sfc.nc', chunks={'time': 2})

In [15]:
ds_ifs_pl

<xarray.Dataset> Size: 35MB
Dimensions:    (longitude: 21, latitude: 21, level: 2, number: 10, time: 248)
Coordinates:
  * longitude  (longitude) float32 84B 75.25 75.5 75.75 ... 79.75 80.0 80.25
  * latitude   (latitude) float32 84B 31.25 31.0 30.75 30.5 ... 26.75 26.5 26.25
  * level      (level) int32 8B 500 850
  * number     (number) int32 40B 1 2 3 4 5 6 7 8 9 10
  * time       (time) datetime64[ns] 2kB 2020-12-03 ... 2024-01-02T12:00:00
Data variables:
    t          (time, number, level, latitude, longitude) float64 17MB dask.array<chunksize=(2, 10, 2, 21, 21), meta=np.ndarray>
    gh         (time, number, level, latitude, longitude) float64 17MB dask.array<chunksize=(2, 10, 2, 21, 21), meta=np.ndarray>

In [16]:
ds_ifs_gh = ds_ifs_pl['gh'].to_dataset("level")
ds_ifs_gh = ds_ifs_gh.rename({500: 'gh_500', 850: 'gh_850'})

In [17]:
ds_ifs_pl

<xarray.Dataset> Size: 35MB
Dimensions:    (longitude: 21, latitude: 21, level: 2, number: 10, time: 248)
Coordinates:
  * longitude  (longitude) float32 84B 75.25 75.5 75.75 ... 79.75 80.0 80.25
  * latitude   (latitude) float32 84B 31.25 31.0 30.75 30.5 ... 26.75 26.5 26.25
  * level      (level) int32 8B 500 850
  * number     (number) int32 40B 1 2 3 4 5 6 7 8 9 10
  * time       (time) datetime64[ns] 2kB 2020-12-03 ... 2024-01-02T12:00:00
Data variables:
    t          (time, number, level, latitude, longitude) float64 17MB dask.array<chunksize=(2, 10, 2, 21, 21), meta=np.ndarray>
    gh         (time, number, level, latitude, longitude) float64 17MB dask.array<chunksize=(2, 10, 2, 21, 21), meta=np.ndarray>

In [18]:
ds_ifs_t = ds_ifs_pl['t'].to_dataset("level")
ds_ifs_t = ds_ifs_t.rename({500: 't_500', 850: 't_850'})

In [19]:
ds_ifs_t['longitude']

<xarray.DataArray 'longitude' (longitude: 21)> Size: 84B
array([75.25, 75.5 , 75.75, 76.  , 76.25, 76.5 , 76.75, 77.  , 77.25, 77.5 ,
       77.75, 78.  , 78.25, 78.5 , 78.75, 79.  , 79.25, 79.5 , 79.75, 80.  ,
       80.25], dtype=float32)
Coordinates:
  * longitude  (longitude) float32 84B 75.25 75.5 75.75 ... 79.75 80.0 80.25
Attributes:
    units:      degrees_east
    long_name:  longitude

In [20]:
ds_ifs_merged = xr.merge([ds_ifs_sfc, ds_ifs_t, ds_ifs_gh])

In [21]:
ds_ifs_merged = ds_ifs_merged.isel(latitude=slice(None, None, -1))
ds_ifs_merged = ds_ifs_merged.rename({'number': 'ensemble'})

In [22]:
ds_ifs_merged.to_netcdf("testout.nc", engine="netcdf4")

PermissionError: [Errno 13] Permission denied: 'd:\\Projects\\NIOT\\v1\\ensemble_transformer\\scripts\\data\\testout.nc'

In [ ]:
ds_ifs_merged[['latitude', 'longitude']]

<xarray.Dataset> Size: 168B
Dimensions:    (latitude: 21, longitude: 21)
Coordinates:
  * latitude   (latitude) float32 84B 26.25 26.5 26.75 27.0 ... 30.75 31.0 31.25
  * longitude  (longitude) float32 84B 75.25 75.5 75.75 ... 79.75 80.0 80.25
Data variables:
    *empty*

In [34]:
ds_ifs_merged[['time']]

<xarray.Dataset> Size: 2kB
Dimensions:  (time: 248)
Coordinates:
  * time     (time) datetime64[ns] 2kB 2020-12-03 ... 2024-01-02T12:00:00
Data variables:
    *empty*

In [32]:
ds_ifs_train = ds_ifs_merged.sel(time=slice('2020-12-01', '2022-12-31'))

In [ ]:
ds_ifs_train

<xarray.Dataset> Size: 32MB
Dimensions:    (longitude: 21, latitude: 21, ensemble: 10, time: 182)
Coordinates:
  * longitude  (longitude) float32 84B 75.25 75.5 75.75 ... 79.75 80.0 80.25
  * latitude   (latitude) float32 84B 26.25 26.5 26.75 27.0 ... 30.75 31.0 31.25
  * ensemble   (ensemble) int32 40B 1 2 3 4 5 6 7 8 9 10
  * time       (time) datetime64[ns] 1kB 2020-12-03 ... 2022-12-31T12:00:00
Data variables:
    t2m        (time, ensemble, latitude, longitude) float64 6MB dask.array<chunksize=(2, 10, 21, 21), meta=np.ndarray>
    t_500      (time, ensemble, latitude, longitude) float64 6MB dask.array<chunksize=(2, 10, 21, 21), meta=np.ndarray>
    t_850      (time, ensemble, latitude, longitude) float64 6MB dask.array<chunksize=(2, 10, 21, 21), meta=np.ndarray>
    gh_500     (time, ensemble, latitude, longitude) float64 6MB dask.array<chunksize=(2, 10, 21, 21), meta=np.ndarray>
    gh_850     (time, ensemble, latitude, longitude) float64 6MB dask.array<chunksize=(2, 10, 21, 21), meta=np.ndarray>

In [23]:
ds_ifs_train = ds_ifs_merged.sel(time=slice('2020-12-01', '2022-12-31'))
ds_ifs_train.to_zarr(
    '../data/processed/ifs/ds_train',
    encoding={
        't2m': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        't_500': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        't_850': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        'gh_500': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        'gh_850': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
    }
)

ContainsGroupError: path '' contains a group

In [43]:
y = ds_ifs_train.sel(time=slice('2021-01-12'))

In [44]:
y

<xarray.Dataset> Size: 11MB
Dimensions:    (longitude: 21, latitude: 21, ensemble: 10, time: 62)
Coordinates:
  * longitude  (longitude) float32 84B 75.25 75.5 75.75 ... 79.75 80.0 80.25
  * latitude   (latitude) float32 84B 26.25 26.5 26.75 27.0 ... 30.75 31.0 31.25
  * ensemble   (ensemble) int32 40B 1 2 3 4 5 6 7 8 9 10
  * time       (time) datetime64[ns] 496B 2020-12-03 ... 2021-01-02T12:00:00
Data variables:
    t2m        (time, ensemble, latitude, longitude) float64 2MB dask.array<chunksize=(2, 10, 21, 21), meta=np.ndarray>
    t_500      (time, ensemble, latitude, longitude) float64 2MB dask.array<chunksize=(2, 10, 21, 21), meta=np.ndarray>
    t_850      (time, ensemble, latitude, longitude) float64 2MB dask.array<chunksize=(2, 10, 21, 21), meta=np.ndarray>
    gh_500     (time, ensemble, latitude, longitude) float64 2MB dask.array<chunksize=(2, 10, 21, 21), meta=np.ndarray>
    gh_850     (time, ensemble, latitude, longitude) float64 2MB dask.array<chunksize=(2, 10, 21, 21), meta=np.ndarray>

In [37]:
ds_ifs_test = ds_ifs_merged.sel(time=slice('2023-12-01', '2023-12-31'))
ds_ifs_test.to_zarr(
    '../data/processed/ifs/ds_test',
    encoding={
        't2m': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        't_500': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        't_850': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        'gh_500': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
        'gh_850': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
    }
)

ContainsGroupError: path '' contains a group

In [38]:
ds_ifs_test[['time']]

<xarray.Dataset> Size: 464B
Dimensions:  (time: 58)
Coordinates:
  * time     (time) datetime64[ns] 464B 2023-12-03 ... 2023-12-31T12:00:00
Data variables:
    *empty*

In [4]:
import xarray as xr
import numpy as np

In [5]:
from typing import Union


In [6]:
z = xr.open_dataset('testout.nc')[['t2m']]

In [9]:
y = xr.open_dataset('out.nc')[['t2m']]

In [11]:
z

<xarray.Dataset> Size: 9MB
Dimensions:    (time: 248, ensemble: 10, latitude: 21, longitude: 21)
Coordinates:
  * longitude  (longitude) float32 84B 75.25 75.5 75.75 ... 79.75 80.0 80.25
  * latitude   (latitude) float32 84B 26.25 26.5 26.75 27.0 ... 30.75 31.0 31.25
  * ensemble   (ensemble) int32 40B 1 2 3 4 5 6 7 8 9 10
  * time       (time) datetime64[ns] 2kB 2020-12-03 ... 2024-01-02T12:00:00
Data variables:
    t2m        (time, ensemble, latitude, longitude) float64 9MB ...

In [17]:
def load_merge_data(era5_dir: str) -> xr.Dataset:
    ds_t2m = xr.open_dataset(
        os.path.join(era5_dir, 't2m_raw.nc')
    ).sel() .chunk({'time': 10})
    
    ds_t850 = xr.open_dataset(
        os.path.join(era5_dir, 't850_raw.nc')
    ).sel() .chunk({'time': 10})
    ds_z500 = xr.open_dataset(
        os.path.join(era5_dir, 'z500_raw.nc')
    ).sel() .chunk({'time': 10})
    print('Got data')
    merged_ds = xr.merge([ds_z500, ds_t850, ds_t2m])
    print('Merged datasets')
    merged_ds = merged_ds.rename({'latitude': 'lat', 'longitude': 'lon'})
    print(merged_ds)
    print('Cleaned data')
    return merged_ds

In [26]:
y = load_merge_data(r"D:\Projects\NIOT\v1\ensemble_transformer\scripts\data\raw")

Got data
Merged datasets
<xarray.Dataset> Size: 657kB
Dimensions:  (lon: 21, lat: 21, time: 62)
Coordinates:
  * lon      (lon) float32 84B 78.0 78.25 78.5 78.75 ... 82.25 82.5 82.75 83.0
  * lat      (lat) float32 84B 15.0 14.75 14.5 14.25 ... 10.75 10.5 10.25 10.0
  * time     (time) datetime64[ns] 496B 2023-12-01 ... 2023-12-31T12:00:00
Data variables:
    z        (time, lat, lon) float64 219kB dask.array<chunksize=(10, 21, 21), meta=np.ndarray>
    t        (time, lat, lon) float64 219kB dask.array<chunksize=(10, 21, 21), meta=np.ndarray>
    t2m      (time, lat, lon) float64 219kB dask.array<chunksize=(10, 21, 21), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-12 07:50:10 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...
Cleaned data


In [27]:
y

<xarray.Dataset> Size: 657kB
Dimensions:  (lon: 21, lat: 21, time: 62)
Coordinates:
  * lon      (lon) float32 84B 78.0 78.25 78.5 78.75 ... 82.25 82.5 82.75 83.0
  * lat      (lat) float32 84B 15.0 14.75 14.5 14.25 ... 10.75 10.5 10.25 10.0
  * time     (time) datetime64[ns] 496B 2023-12-01 ... 2023-12-31T12:00:00
Data variables:
    z        (time, lat, lon) float64 219kB dask.array<chunksize=(10, 21, 21), meta=np.ndarray>
    t        (time, lat, lon) float64 219kB dask.array<chunksize=(10, 21, 21), meta=np.ndarray>
    t2m      (time, lat, lon) float64 219kB dask.array<chunksize=(10, 21, 21), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-12 07:50:10 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [29]:
ds_ifs_merged = 

In [28]:
ds_ifs_merged = ds_ifs_merged.rename({'latitude': 'lat', 'longitude': 'lon'})

In [29]:
def get_regridder(grid_in: xr.Dataset, out_res: float, method='bilinear'):
    import xesmf as xe
    grid_out = xr.Dataset(
        {
            'lat': (['lat'], np.arange(-90+out_res/2, 90, out_res)),
            'lon': (['lon'], np.arange(0, 360, out_res)),
        }
    )
    regridder = xe.Regridder(grid_in, grid_out, method=method,
                             periodic=True, reuse_weights=False, ignore_degenerate=False)
    return regridder


def _regrid_loop(
        regridder,
        ds: xr.Dataset,
        chunk_size: Union[int, None] = None
) -> xr.Dataset:
    if chunk_size is None:
        chunk_size = len(ds['time'])
    ds_regridded = []
    regrid_pbar = range(0, len(ds['time']), chunk_size)
    for chunk in regrid_pbar:
        tmp_regridded = regridder(
            ds.isel(time=slice(chunk, chunk+chunk_size))
        )
        ds_regridded.append(tmp_regridded)
    ds_regridded = xr.concat(ds_regridded, dim='time')
    for var in ds_regridded:
        ds_regridded[var].attrs = ds[var].attrs
    ds_regridded.attrs = ds.attrs
    return ds_regridded


def regrid_ds(
        ds_ens: Union[xr.Dataset, None],
        chunk_size: Union[int, None] = None
) -> Union[xr.Dataset, None]:
    if ds_ens is not None:
        grid_in = ds_ens[['lat', 'lon']]
        print(grid_in)
        regridder = get_regridder(grid_in, out_res=5.625)
        ds_regridded = _regrid_loop(regridder, ds_ens, chunk_size)
    else:
        ds_regridded = None
    return ds_regridded

In [30]:
y = regrid_ds(ds_ifs_merged, chunk_size=100)

<xarray.Dataset> Size: 168B
Dimensions:  (lat: 21, lon: 21)
Coordinates:
  * lat      (lat) float32 84B 10.0 10.25 10.5 10.75 ... 14.25 14.5 14.75 15.0
  * lon      (lon) float32 84B 78.0 78.25 78.5 78.75 ... 82.25 82.5 82.75 83.0
Data variables:
    *empty*
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-18 15:14:25 GMT by grib_to_netcdf-2.33.2: /opt/ecmw...


In [31]:
ds_ifs_merged = y

In [32]:
ds_ifs_merged

<xarray.Dataset> Size: 51MB
Dimensions:   (time: 62, ensemble: 10, lat: 32, lon: 64)
Coordinates:
  * ensemble  (ensemble) int32 40B 1 2 3 4 5 6 7 8 9 10
  * time      (time) datetime64[ns] 496B 2023-12-03 ... 2024-01-02T12:00:00
  * lat       (lat) float64 256B -87.19 -81.56 -75.94 ... 75.94 81.56 87.19
  * lon       (lon) float64 512B 0.0 5.625 11.25 16.88 ... 343.1 348.8 354.4
Data variables:
    t2m       (time, ensemble, lat, lon) float64 10MB dask.array<chunksize=(2, 10, 32, 64), meta=np.ndarray>
    t_500     (time, ensemble, lat, lon) float64 10MB dask.array<chunksize=(2, 10, 32, 64), meta=np.ndarray>
    t_850     (time, ensemble, lat, lon) float64 10MB dask.array<chunksize=(2, 10, 32, 64), meta=np.ndarray>
    gh_500    (time, ensemble, lat, lon) float64 10MB dask.array<chunksize=(2, 10, 32, 64), meta=np.ndarray>
    gh_850    (time, ensemble, lat, lon) float64 10MB dask.array<chunksize=(2, 10, 32, 64), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-18 15:14:25 GMT by grib_to_netcdf-2.33.2: /opt/ecmw...

In [33]:
ds_ifs_merged = ds_ifs_merged.rename({'lat': 'latitude', 'lon': 'longitude'})

In [34]:
ds_ifs_merged

<xarray.Dataset> Size: 51MB
Dimensions:    (time: 62, ensemble: 10, latitude: 32, longitude: 64)
Coordinates:
  * ensemble   (ensemble) int32 40B 1 2 3 4 5 6 7 8 9 10
  * time       (time) datetime64[ns] 496B 2023-12-03 ... 2024-01-02T12:00:00
  * latitude   (latitude) float64 256B -87.19 -81.56 -75.94 ... 81.56 87.19
  * longitude  (longitude) float64 512B 0.0 5.625 11.25 ... 343.1 348.8 354.4
Data variables:
    t2m        (time, ensemble, latitude, longitude) float64 10MB dask.array<chunksize=(2, 10, 32, 64), meta=np.ndarray>
    t_500      (time, ensemble, latitude, longitude) float64 10MB dask.array<chunksize=(2, 10, 32, 64), meta=np.ndarray>
    t_850      (time, ensemble, latitude, longitude) float64 10MB dask.array<chunksize=(2, 10, 32, 64), meta=np.ndarray>
    gh_500     (time, ensemble, latitude, longitude) float64 10MB dask.array<chunksize=(2, 10, 32, 64), meta=np.ndarray>
    gh_850     (time, ensemble, latitude, longitude) float64 10MB dask.array<chunksize=(2, 10, 32, 64), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2024-06-18 15:14:25 GMT by grib_to_netcdf-2.33.2: /opt/ecmw...